<a href="https://colab.research.google.com/github/MarMarhoun/freelance_work/blob/main/side_projects/NLP_projs/eda_streamlit/face_age_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face and age detection using online dataset, streamlit and tensorflow

The code for face and age detection using online dataset, Streamlit, and TensorFlow, I would suggest incorporating an age detection model in addition to the existing face detection and verification functionality. You can use the Indico's "age-detection-1" model, which is available in their TensorFlow model library.

First, install the Indico library and download the age detection model:

In [ ]:
pip install indicoio
indico download --models age-detection-1

Now, update the requirements.txt file with the new library:



In [ ]:
...
indicoio
keras-vggface
...

Next, modify the app.py file to include age detection functionality:



In [ ]:
import ...
import indicoio
from indicoio.utils import download_model

# Download the age detection model if not present
model_path = download_model('age-detection-1')
indicoio.set_concurrency(1)
indicoio.config.update({'api_key': 'YOUR_API_KEY', 'age_detection_model_path': model_path})

# ...

def predict_age(image):
    results = indicoio.age_detection(image)
    return results['ages'][0], results['ages'][1]

# ...

elif choice == "Face Detection 2":
    # ...
    with column2:
        if age_prediction:
            min_age, max_age = predict_age(face_array)
            st.write(f"Predicted Age: {min_age}-{max_age} years")
        plt.imshow(face_array)
        st.pyplot(fig)

# ...

elif choice == "Face Verification":
    # ...
    if st.button("Verify Faces"):
        if score <= thresh:
            st.success(" > Faces are a match (%.3f <= %.3f)" % (score, thresh))
            if age_prediction:
                min_age1, max_age1 = predict_age(faces[0])
                min_age2, max_age2 = predict_age(faces[1])
                st.write(f"Person 1 Age: {min_age1}-{max_age1} years")
                st.write(f"Person 2 Age: {min_age2}-{max_age2} years")
        else:
            st.error(" > Faces are NOT a match (%.3f > %.3f)" % (score, thresh))

# ...

# Add this to the UI to enable/disable age prediction
age_prediction = st.checkbox("Enable Age Prediction")

Make sure to replace 'YOUR_API_KEY' with your actual Indico API key.

This updated code will now provide age predictions for the detected faces in the "Face Detection 2" and "Face Verification" options.

To enhance the code for face and age detection using online dataset, Streamlit, and TensorFlow, we can add an age detection feature to the existing face detection application. We can use the MTCNN model for face detection and the InsightFace model for age detection. Here's an example of how to implement this feature:

First, we need to install the required packages:

In [ ]:
pip install mtcnn
pip install opencv-python
pip install tensorflow
pip install keras
pip install streamlit
pip install git+https://github.com/deepinsight/insightface.git

Then, we can create a new Python file (e.g., face_age_detection.py) and add the following code:

In [ ]:
import streamlit as st
import cv2
import numpy as np
import tensorflow as tf
from mtcnn import MTCNN
from insightface.app import FaceAnalysis

# Initialize face detection and age detection models
mtcnn = MTCNN()
face_analysis = FaceAnalysis(providers=['CUDA( Tesla K80 )'])

# Define a function to detect faces and estimate ages
def detect_faces_and_estimate_ages(image):
    # Detect faces using MTCNN
    faces = mtcnn.detect_faces(image)

    # Initialize an empty list to store face embeddings and ages
    embeddings = []
    ages = []

    # Iterate over the detected faces
    for face in faces:
        # Extract face region
        x, y, w, h = face['box']
        face_image = image[y:y+h, x:x+w]

        # Detect face using InsightFace
        result = face_analysis.analysis(face_image)

        # Extract face embedding and age
        embedding = result[' face_embeddings'][0]
        age = int(result['age'][0]['mean'])

        # Append the embedding and age to the respective lists
        embeddings.append(embedding)
        ages.append(age)

    return embeddings, ages

# Define a function to draw bounding boxes and display ages
def draw_boxes_and_display_ages(image, faces, ages):
    # Loop over the detected faces
    for i, face in enumerate(faces):
        # Get the bounding box coordinates
        x, y, w, h = face['box']

        # Draw a bounding box around the face
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Display the age above the bounding box
        cv2.putText(image, str(ages[i]), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Display the image
    st.image(image, channels="BGR")

# Define the main function
def main():
    # Create a file uploader
    uploaded_file = st.file_uploader("Choose an image", type="jpg")

    # If an image is uploaded, display it with face detection and age estimation
    if uploaded_file is not None:
        # Load the image
        image = np.frombuffer(uploaded_file.read(), np.uint8)
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)

        # Detect faces and estimate ages
        embeddings, ages = detect_faces_and_estimate_ages(image)

        # Draw bounding boxes and display ages
        draw_boxes_and_display_ages(image, embeddings, ages)

# Run the main function
if __name__ == "__main__":
    main()

This code creates a Streamlit application that allows users to upload an image and displays the image with bounding boxes around the detected faces and the estimated ages above each bounding box.

Note: This is just an example of how to add an age detection feature to the existing face detection application. The accuracy of age detection may vary depending on the dataset and the model used.

I will now provide the age detection function and integrate it with the existing face detection code using Streamlit. You can utilize the pre-trained model from the PyImageSearch repository.

First, download the age detection models:

In [ ]:
!wget https://github.com/AdrianRosebrock/deep-learning-face-detection/releases/download/v0.0.6/age_detector.zip
unzip age_detector.zip
rm age_detector.zip

Now, update your app.py:



In [ ]:
import streamlit as st
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import numpy as np
import cv2
import os

# Define functions
def detect_faces(image, face_model):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_model.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

def annotate_faces(image, faces, draw):
    for (x, y, w, h) in faces:
        draw.rectangle([x, y, x+w, y+h], outline=(255, 0, 0), width=2)

def detect_age(image, age_bucket_model, age_model_path):
    age_buckets = ["(0-2)", "(4-6)", "(8-12)", "(15-20)", "(25-32)", "(38-43)", "(48-53)", "(60-100)"]
    age_bucket_model.setInput(cv2.dnn.blobFromImage(image, 1.0, (227, 227), mean=(78.4263377603, 87.7689143744, 84.7034765625), swapRB=False, crop=False))
    age_output = age_bucket_model.forward()
    age = age_buckets[np.argmax(age_output)]
    return age

# Initialize models
face_model = cv2.dnn.readNetFromCaffe("face_detector/deploy.prototxt", "face_detector/res10_300x300_ssd_iter_140000.caffemodel")
age_bucket_model = cv2.dnn.readNetFromCaffe(os.path.join("age_detector", "age_deploy.prototxt"), os.path.join("age_detector", "age_net.caffemodel"))

# Streamlit app
st.set_page_config(page_title="Face and Age Detection", page_icon=":guardsman:", layout="wide")
st.title("Face and Age Detection")

uploaded_file = st.file_uploader("Upload an image", type="jpg, jpeg, png")

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded Image", use_column_width=True)

    # Detect faces and draw rectangles
    img_array = np.array(image)
    faces = detect_faces(img_array, face_model)
    draw = ImageDraw.Draw(image)
    annotate_faces(image, faces, draw)

    # Detect age and display
    age = detect_age(img_array, age_bucket_model, "age_detector")
    st.write("Detected Age:", age)

    # Display image with annotations
    st.image(image, caption="Detected Faces and Age", use_column_width=True)

This updated Streamlit code includes the age detection function and integrates it with the existing face detection code. You can use this code as a base for further enhancements.

